In [1]:
import pandas as pd
from lifelines import CoxPHFitter
from sklearn.model_selection import train_test_split
import numpy as np

#### About

Baseline Performance of an Oracle Cox Proportional Hazards model

In [2]:
df = pd.read_csv("data/mimic3_df_30.csv", index_col=[0,1])

In [3]:
df_sub = df.drop(
    columns=["treated", "control", "hazard", "q", "survival_prob", 
             "survives", "censored","corrected_survival", "critical", "first_failure",
            "baseline_hazard"]
)
df_flat = df_sub.groupby(level=0).mean()

In [4]:
df_flat["total_hours"] = df.groupby(level=0)["corrected_survival"].sum()
df_flat["uncensored"] = (df.groupby(level=0)["corrected_survival"].min() == 0).astype(int)

In [5]:
def mae(df, y_hat):
    a = np.abs((df["total_hours"] - y_hat)[df["uncensored"].astype(bool)]).sum()
    b = np.maximum(np.zeros(df.shape[0]), df["total_hours"] - y_hat).sum()
    return (a + b) / df.shape[0]

In [6]:
df_flat.head()

,gender,stay_length,hypertension,coronary_ath,atrial_fib,hematocrit,hemoglobin,platelets,mean blood pressure,A,total_hours,uncensored
subject_id,,,,,,,,,,,,
4,0.0,-0.360949,0.0,0.0,0.0,0.036834,-0.079433,0.041209,0.740383,0.0,41,0
6,0.0,1.359324,0.0,0.0,0.0,0.476099,0.348309,0.183846,0.698708,0.0,8,1
9,1.0,2.757045,1.0,0.0,0.0,1.195403,1.210150,0.519788,1.060392,0.0,11,1
11,0.0,-0.432627,0.0,0.0,0.0,1.060155,1.016993,0.348131,-0.467836,0.0,39,0
13,0.0,1.323485,1.0,1.0,0.0,-0.087935,-0.180417,-0.808778,0.477129,0.0,44,1


In [7]:
maes = []
cis = []
for seed in [71, 72, 73, 74, 75, 76, 77]:
    print(seed)
    train, rest = train_test_split(df_flat, train_size=0.7, random_state=seed)
    val, test = train_test_split(rest, test_size=0.5, random_state=seed)
    val_scores = []
    val_cis = []
    test_scores = []
    test_cis = []
    for lam in [0, .1, .2,]:
        for r in [.1, .2]:
            cph = CoxPHFitter(penalizer=lam, l1_ratio=r)
            cph.fit(train, duration_col="total_hours", event_col="uncensored")
            y_hat_val = cph.predict_expectation(val)
            val_scores.append(mae(val, y_hat_val))
            y_hat_test = cph.predict_expectation(test)
            test_scores.append(mae(test, y_hat_test))
            val_cis.append(cph.score(val, "concordance_index"))
            test_cis.append(cph.score(test, "concordance_index"))
    best_ix = np.argmin(val_scores)
    best2 = np.argmin(test_scores)
    maes.append(test_scores[best_ix])
    cis.append(test_cis[best2])

71
72
73
74
75
76
77


In [12]:
val_scores

[16.382231334035115,
 16.382231334035115,
 16.055424052817813,
 16.050853797460622,
 16.107690687223034,
 16.248261454003153]

In [8]:
maes

[16.152028999146633,
 16.21436190365927,
 16.135267209966496,
 15.44158683598311,
 16.156144117906955,
 16.045681186467082,
 16.159166149032547]

In [9]:
cis

[0.7255777850280652,
 0.726912349257252,
 0.7203486264726358,
 0.7331455904125083,
 0.7224752386630594,
 0.7241564587502033,
 0.7243051224904751]

In [11]:
np.mean(maes), np.std(maes)

(16.043462343166013, 0.25007573555650764)